In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:4 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:5 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:6 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:11 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [1,057 kB]
Get:12 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,526 kB]
Get:13 http://security.ubuntu.com/ubuntu bionic-security/main a

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-07-16 15:58:26--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  1.61MB/s    in 0.6s    

2022-07-16 15:58:27 (1.61 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
#Config part adds the driver to spark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Review-Analysis-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://amazon-vine-analysis-challenge.s3.us-east-2.amazonaws.com/amazon_reviews_us_Mobile_Electronics_v1_00.tsv"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Mobile_Electronics_v1_00.tsv"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|  product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   20422322| R8MEA6IGAHO0B|B00MC4CED8|     217304173|BlackVue DR600GW-PMP|Mobile_Electronics|          5|            0|          0|   N|                Y|         Very Happy!|As advertised. Ev...| 2015-08-31|
|         US|   40835037|R31LOQ8JGLPRLK|B00OQMFG1Q|     137313254|GENSSI GSM / GPS ...|Mobile_Electronics|      

In [5]:
#Retrieve all rows where total_votes is equal or greater than 20
df_filtered = df.filter(df.total_votes>=20)
df_filtered.show()

+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|  product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   48701722| RBEMQ29WJBHYG|B00YO3UYXW|     913911909|Jensen MCR-100 Ca...|Mobile_Electronics|          5|          164|        168|   N|                Y|  I LOVE my recorder|I LOVE my recorde...| 2015-08-31|
|         US|   40963441|R2JK5Y8D5MXAGP|B00SU7B0JK|      82850235|iXCC Multi pack L...|Mobile_Electronics|      

In [7]:
# Filter the created df to retrieve all rows where helpful_votes/total_votes is equal or greater than 50%
df_helpful_votes_rate = df_filtered.filter((df.helpful_votes/df.total_votes) >=0.5)
df_helpful_votes_rate.show()

+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|  product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   48701722| RBEMQ29WJBHYG|B00YO3UYXW|     913911909|Jensen MCR-100 Ca...|Mobile_Electronics|          5|          164|        168|   N|                Y|  I LOVE my recorder|I LOVE my recorde...| 2015-08-31|
|         US|   25750755|R13W3EMIUV120L|B00YO3UYXW|     913911909|Jensen MCR-100 Ca...|Mobile_Electronics|      

In [9]:
#Filter rows where the review was written as part of the Vine program (paid)
df_vine = df_helpful_votes_rate.filter(df_helpful_votes_rate.vine =="Y")
df_vine.show()

+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|  product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   52490988|R3BOQTLUU3Y4L8|B00EP0SIOG|     628459233|Orbotix Sphero 2....|Mobile_Electronics|          4|           42|         55|   Y|                N|Lot of fun, but f...|Before receiving ...| 2013-11-19|
|         US|   48814959| RWF03LXVXC22A|B00EP0SIOG|     628459233|Orbotix Sphero 2....|Mobile_Electronics|      

In [10]:
#Filter rows where the review was not part of the Vine program (unpaid)
df_no_vine = df_helpful_votes_rate.filter(df_helpful_votes_rate.vine=="N")
df_no_vine.show()

+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|  product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   48701722| RBEMQ29WJBHYG|B00YO3UYXW|     913911909|Jensen MCR-100 Ca...|Mobile_Electronics|          5|          164|        168|   N|                Y|  I LOVE my recorder|I LOVE my recorde...| 2015-08-31|
|         US|   25750755|R13W3EMIUV120L|B00YO3UYXW|     913911909|Jensen MCR-100 Ca...|Mobile_Electronics|      

In [27]:
# Total number of review for paid and unpaid
count_paid_reviews=df_vine.count()
print('Total number of paid reviews:', count_paid_reviews)
count_unpaid_reviews=df_no_vine.count()
print('Total number of unpaid reviews:', count_unpaid_reviews)
total_reviews=count_paid_reviews+count_unpaid_reviews
print('Total number of reviews is:',total_reviews)
print("")

# Total 5-star reviews for paid and unpaid
count_paid_5_stars = df_vine.filter(df_vine.star_rating == 5).count()
print('Total number of 5-stars reviews paid:', count_paid_5_stars)
count_unpaid_5_stars = df_no_vine.filter(df_no_vine.star_rating == 5).count()
print('Total number of 5-stars reviews unpaid:', count_unpaid_5_stars)
print('Total number of 5-stars reviews',count_paid_5_stars+count_unpaid_5_stars)
print("")

# Percentage of 5-star reviews for paid and unpaid
percentage_paid_5_stars = count_paid_5_stars/count_paid_reviews*100
print("Percentage of 5-stars-reviews paid per total paid reviews",round(percentage_paid_5_stars,2),'%')
percentage_unpaid_5_stars = count_unpaid_5_stars/count_unpaid_reviews*100
print("Percentage of 5-stars-reviews unpaid per total unpaid reviews",round(percentage_unpaid_5_stars,2),'%')

Total number of paid reviews: 4
Total number of unpaid reviews: 1064
Total number of reviews is: 1068

Total number of 5-stars reviews paid: 1
Total number of 5-stars reviews unpaid: 527
Total number of 5-stars reviews 528

Percentage of 5-stars-reviews paid per total paid reviews 25.0 %
Percentage of 5-stars-reviews unpaid per total unpaid reviews 49.53 %
